In [1]:
!pip install -U transformers accelerate peft bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

In [2]:
!nvidia-smi

Thu May  8 09:54:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
TARGET_MODEL = "deepseek-ai/deepseek-math-7b-base"

DEBUG = False

In [4]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch
import numpy  as np
import random
import os
from sklearn.model_selection import StratifiedKFold , GroupKFold , KFold

2025-05-08 09:54:35.357557: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746698075.554783      18 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746698075.610688      18 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(42)

In [6]:
import pandas as pd
train_df = pd.read_csv('/kaggle/input/math-reasoning-datasets/train.csv')

In [7]:
train_df.head()

,Question,label
0,[QUESTION] : A solitaire game is played as fol...,3
1,[QUESTION] : 2. The school table tennis champi...,5
2,"[QUESTION] : Given that $x, y,$ and $z$ are re...",0
3,[QUESTION] : $25 \cdot 22$ Given three distinc...,1
4,[QUESTION] : I am thinking of a five-digit num...,5


In [8]:
class CFG : 
    n_folds = 5

cfg = CFG()

In [9]:
kfolds = StratifiedKFold(n_splits=cfg.n_folds, shuffle=True, random_state=42)

train_df['fold'] = -1
for fold_num, (train_idxs, test_idxs) in enumerate(kfolds.split(train_df.index, train_df["label"].values)):
    train_df.loc[test_idxs, ['fold']] = fold_num

In [10]:
# load model with 4bit bnb

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch

In [11]:
from huggingface_hub import login

# Replace 'your_hf_token' with your actual Hugging Face access token
login("'''J")

In [12]:
from transformers import AutoTokenizer, LlamaForSequenceClassification , AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

In [13]:
special_tokens_dict = {
    "additional_special_tokens": ["[QUESTION] :", "[REASONING] :"]
}
tokenizer.add_special_tokens(special_tokens_dict)

text = "[QUESTION] : A triangular array of squares has one square in the first row... [REASONING] : The problem is about..."
tokens = tokenizer(text)
print("Number of tokens:", len(tokens['input_ids']))
print(f'tokens : {tokens}')
encoded = tokenizer(text)
input_ids = encoded['input_ids']
reconstructed_text = tokenizer.decode(input_ids, skip_special_tokens=False)
reconstructed_text

Number of tokens: 22
tokens : {'input_ids': [100000, 100002, 338, 35896, 3869, 280, 26532, 643, 634, 7742, 279, 254, 1022, 5451, 1204, 207, 100003, 429, 2066, 317, 786, 1204], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


'<｜begin▁of▁sentence｜>[QUESTION] : A triangular array of squares has one square in the first row... [REASONING] : The problem is about...'

In [14]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        
        all_param += param.numel()
        if param.requires_grad:
            print(_)
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [15]:
def preprocess_function(examples, max_length=512):
    return tokenizer(examples["input"], truncation=True, max_length=max_length, padding=True)

In [16]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [17]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score , f1_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    f1_score_val = f1_score(labels, predictions, average="micro")
    
    return {
        "f1_score": f1_score_val,
    }

In [18]:
def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

In [19]:
train = train_df.copy()

In [20]:
train.rename(columns = {'Question' : "input"} , inplace = True)

In [21]:
from transformers import TrainingArguments, Trainer
from datasets import Dataset

steps = 5 if DEBUG else 50
for fold in [0]:
    print(f'Fold {fold+1}/5')

    valid_df = train[train["fold"] == fold]
    train_df = train[train["fold"] != fold]
    
    # from pandas
    train_ds = Dataset.from_pandas(train_df)
    valid_ds = Dataset.from_pandas(valid_df)
    
    train_tokenized_ds = train_ds.map(preprocess_function, batched=True)
    valid_tokenized_ds = valid_ds.map(preprocess_function, batched=True)
    
    # Model
    
    peft_config = LoraConfig(
        r=64,
        lora_alpha=128,
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        target_modules=[
    "q_proj",
    "v_proj",
    "k_proj",
    "out_proj",
    "fc1",
    "fc2",
    "dense"
]
,
    )

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    
    base_model = AutoModelForSequenceClassification.from_pretrained(
        TARGET_MODEL,
        num_labels=8,
        quantization_config=bnb_config,
        device_map={"":0}
    )
    base_model.config.pretraining_tp = 1 # 1 is 7b
    base_model.config.pad_token_id = tokenizer.pad_token_id
    
    # Load the PEFT model from checkpoint
    checkpoint_path = f"/kaggle/input/math-reasoning-seek-fold-0-token/outputs/fold{fold}/checkpoint-1018" 
    model = PeftModel.from_pretrained(base_model, checkpoint_path)
    
    training_args = TrainingArguments(
        output_dir=f"outputs/fold{fold}",
        learning_rate=1e-4,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=8,
        max_grad_norm= 3,#0.3,
        optim='paged_adamw_32bit',
        lr_scheduler_type="cosine",
        num_train_epochs=1,
        weight_decay=0.0001,
        eval_strategy="steps",
        save_strategy="steps",
        save_steps = 300,
        eval_steps = 300,
        logging_steps= 300,
        load_best_model_at_end=True,
        push_to_hub=False,
        warmup_steps=10,
        report_to='none' # if DEBUG else 'wandb',
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized_ds,
        eval_dataset=valid_tokenized_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    
    trainer.train()
    # validation 
    pred_output = trainer.predict(valid_tokenized_ds)
    logits = pred_output.predictions
    probas = softmax(logits)
    np.save(f'fold{fold}.npy', probas)

Fold 1/5


Map:   0%|          | 0/8151 [00:00<?, ? examples/s]

Map:   0%|          | 0/2038 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at deepseek-ai/deepseek-math-7b-base and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_18/1127516837.py:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss,F1 Score
300,0.185600,0.264160,0.895486
600,0.123600,0.260498,0.901374
900,0.184000,0.258789,0.901865
